In [1]:
import fitsio
import numpy as np
import pandas as pd
from astropy.coordinates import SkyCoord, get_moon, EarthLocation
from astropy.time import Time
from astropy.io.fits.hdu.hdulist import HDUList
from datetime import date
from astral import Astral, Location
from jdutil import mjd_to_jd, jd_to_date

In [5]:
def get_var(filename):
    #getting RA and DEC
    RA=[]
    DEC=[]
    for i in files:
        sampling=fitsio.read_header(i)
        RA.append(sampling['RA'])
        DEC.append(sampling['DEC'])
    #FINDING INTERPLATE SKY TIME
    fin_mean=[] #time for each moon sky observation interplate, used for moon angle
    for r in range(len(files)): #mechanization of ubiquitious exposure count
        sampling=fitsio.read_header(files[r],0) #finding number of exposures
        h_beg=[]
        h_end=[]
        mean_per=[] #time for each moon sky observation intraplate
        for k in range(4, sampling['NEXP']+4): #4 is constant for everything
            h=fitsio.read_header(files[3],k)
            h_beg.append(h['TAI-BEG']) #ONLY VARIES WITH PLATE NUMBER, different for k's
            h_end.append(h['TAI-END']) #ONLY VARIES WITH PLATE NUMBER, different for k's
            mean_times=(h['TAI-BEG']+h['TAI-END'])/2
            mean_per.append(mean_times)
        tot=np.mean(mean_per)
        fin_mean.append(tot)
    #TAI TO MJD
    new_time=np.array(fin_mean)
    time_MJD=new_time/(86400)
    moon_coords1=[]
    moon_coords=[]
    for i in range (len(time_MJD)):
        t=Time(time_MJD[i], format='mjd')
        moon_coords1.append(get_moon(t))
        moon_coords.append(moon_coords1[i].spherical)
    moon_coords=list(moon_coords)
    #print(moon_coords)
    moon_coords2=[]
    for i in range(len(moon_coords)):
        moon_coords2.append(moon_coords[i]._values)
    moon_coords2=np.array(moon_coords2)
    sky_coords=np.vstack((RA,DEC)).T
    #SPLITTING ARRAY
    new_sky_coords=sky_coords
    new_sky_RA = [item[0] for item in new_sky_coords]
    new_sky_DEC = [item[1] for item in new_sky_coords]
    moon_RA = [item[0] for item in moon_coords2]
    moon_DEC = [item[1] for item in moon_coords2]
    new_sky_RA=np.array(new_sky_RA)*(np.pi/180)
    new_sky_DEC=np.array(new_sky_DEC)*(np.pi/180)
    moon_RA=np.array(moon_RA)*(np.pi/180)
    moon_DEC=np.array(moon_DEC)*(np.pi/180)
    #MOON-SKY ANGLE CALCULATIONS
    moon_sky_angle=[]
    for i in range(len(new_sky_coords)):
        moon_sky_angle.append(np.arccos((np.sin(new_sky_RA[i])*np.sin(moon_RA[i]))+np.cos(new_sky_RA[i])*np.cos(moon_RA[i])*(np.cos(new_sky_DEC[i]-moon_DEC[i])))*(180/np.pi))
    #setting up MJD format
    con1=[]
    date_of_ob=[] #Actual dates
    for i in range(len(files)):
        con1.append(mjd_to_jd(time_MJD[i]))
        date_of_ob.append(jd_to_date(con1[i]))
    year = [item[0] for item in date_of_ob]
    month = [item[1] for item in date_of_ob]
    day = [item[2] for item in date_of_ob]
    d=[]
    for i in range(len(files)):
        d.append(date(int(year[i]),int(month[i]),int(day[i])))
    #giving location
    l = Location()
    l.name = 'Apache Point Observatory'
    l.region = 'NM'
    l.latitude = 32.780208
    l.longitude = -105.819749
    l.timezone = 'US/Mountain'
    l.elevation = 2790
    #calculating moon phase
    moon_phases=[]
    for i in range(len(files)):
        moon_phases.append(l.moon_phase(date=d[i]))
    #0=New moon
    #7=First quarter
    #14=Full moon
    #21=Last quarter
    moon_phases_array=np.array(moon_phases)
    phase=[]
    for i in moon_phases_array:
        if i == 0:
            phase.append(0)
        if i == 7:
            phase.append(0.5)
        if i == 14:
            phase.append(1)
        if i == 21:
            phase.append(0.5)
    var=[]
    for j in range(len(moon_sky_angle)):
        var.append(np.cos(moon_sky_angle[j])*phase[j])
    return(var)

In [6]:
files=('spec-3587-55182-0196.fits','spec-3586-55181-0218.fits','spec-10000-57346-0012.fits', 'spec-10000-57346-0018.fits', 'spec-10000-57346-0028.fits', 'spec-10000-57346-0034.fits', 'spec-10000-57346-0036.fits', 'spec-10000-57346-0044.fits', 'spec-10000-57346-0046.fits', 'spec-10000-57346-0052.fits', 'spec-10000-57346-0064.fits', 'spec-10000-57346-0072.fits', 'spec-10000-57346-0080.fits', 'spec-10000-57346-0082.fits', 'spec-10000-57346-0085.fits', 'spec-10000-57346-0086.fits', 'spec-10000-57346-0105.fits', 'spec-10000-57346-0109.fits', 'spec-10000-57346-0113.fits', 'spec-10000-57346-0128.fits', 'spec-10000-57346-0130.fits', 'spec-10000-57346-0132.fits', 'spec-10000-57346-0149.fits', 'spec-10000-57346-0164.fits', 'spec-10000-57346-0170.fits', 'spec-10000-57346-0173.fits', 'spec-10000-57346-0184.fits', 'spec-10000-57346-0205.fits', 'spec-10000-57346-0206.fits', 'spec-10000-57346-0217.fits', 'spec-10000-57346-0225.fits', 'spec-10000-57346-0242.fits', 'spec-10000-57346-0249.fits', 'spec-10000-57346-0258.fits', 'spec-10000-57346-0272.fits', 'spec-10000-57346-0281.fits', 'spec-10000-57346-0286.fits', 'spec-10000-57346-0289.fits', 'spec-10000-57346-0313.fits', 'spec-10000-57346-0324.fits', 'spec-10000-57346-0330.fits', 'spec-10000-57346-0334.fits', 'spec-10000-57346-0348.fits', 'spec-10000-57346-0366.fits', 'spec-10000-57346-0367.fits', 'spec-10000-57346-0375.fits', 'spec-10000-57346-0398.fits', 'spec-10000-57346-0402.fits', 'spec-10000-57346-0408.fits', 'spec-10000-57346-0418.fits', 'spec-10000-57346-0440.fits', 'spec-10000-57346-0444.fits', 'spec-10000-57346-0446.fits', 'spec-10000-57346-0453.fits', 'spec-10000-57346-0462.fits', 'spec-10000-57346-0486.fits', 'spec-10000-57346-0503.fits', 'spec-10000-57346-0506.fits', 'spec-10000-57346-0518.fits', 'spec-10000-57346-0538.fits', 'spec-10000-57346-0548.fits', 'spec-10000-57346-0550.fits', 'spec-10000-57346-0558.fits', 'spec-10000-57346-0564.fits', 'spec-10000-57346-0587.fits', 'spec-10000-57346-0594.fits', 'spec-10000-57346-0600.fits', 'spec-10000-57346-0609.fits', 'spec-10000-57346-0626.fits', 'spec-10000-57346-0634.fits', 'spec-10000-57346-0636.fits', 'spec-10000-57346-0659.fits', 'spec-10000-57346-0665.fits', 'spec-10000-57346-0672.fits', 'spec-10000-57346-0677.fits', 'spec-10000-57346-0691.fits', 'spec-10000-57346-0704.fits', 'spec-10000-57346-0716.fits', 'spec-10000-57346-0717.fits', 'spec-10000-57346-0735.fits', 'spec-10000-57346-0750.fits', 'spec-10000-57346-0768.fits', 'spec-10000-57346-0782.fits', 'spec-10000-57346-0786.fits', 'spec-10000-57346-0792.fits', 'spec-10000-57346-0812.fits', 'spec-10000-57346-0823.fits', 'spec-10000-57346-0826.fits', 'spec-10000-57346-0832.fits', 'spec-10000-57346-0852.fits', 'spec-10000-57346-0861.fits', 'spec-10000-57346-0868.fits', 'spec-10000-57346-0871.fits', 'spec-10000-57346-0895.fits', 'spec-10000-57346-0905.fits', 'spec-10000-57346-0908.fits', 'spec-10000-57346-0928.fits', 'spec-10000-57346-0929.fits', 'spec-10000-57346-0934.fits', 'spec-10000-57346-0955.fits', 'spec-10000-57346-0956.fits', 'spec-10000-57346-0958.fits', 'spec-10000-57346-0964.fits', 'spec-10000-57346-0965.fits')

In [7]:
get_var(files)

[-0.12479730276056686,
 0.32893184409365439,
 -0.17198439018467565,
 -0.17198439018467565,
 -0.17198439018467565,
 -0.17198439018467565,
 -0.17198439018467565,
 -0.17198439018467565,
 -0.17198439018467565,
 -0.17198439018467565,
 -0.17198439018467565,
 -0.17198439018467565,
 -0.17198439018467565,
 -0.17198439018467565,
 -0.17198439018467565,
 -0.17198439018467565,
 -0.17198439018467565,
 -0.17198439018467565,
 -0.17198439018467565,
 -0.17198439018467565,
 -0.17198439018467565,
 -0.17198439018467565,
 -0.17198439018467565,
 -0.17198439018467565,
 -0.17198439018467565,
 -0.17198439018467565,
 -0.17198439018467565,
 -0.17198439018467565,
 -0.17198439018467565,
 -0.17198439018467565,
 -0.17198439018467565,
 -0.17198439018467565,
 -0.17198439018467565,
 -0.17198439018467565,
 -0.17198439018467565,
 -0.17198439018467565,
 -0.17198439018467565,
 -0.17198439018467565,
 -0.17198439018467565,
 -0.17198439018467565,
 -0.17198439018467565,
 -0.17198439018467565,
 -0.17198439018467565,
 -0.17198439

In [22]:
sampling=fitsio.read_header(files[2])
sampling['RA']

29.815837

In [23]:
RA=[]
DEC=[]
for i in files:
    sampling=fitsio.read_header(i)
    RA.append(sampling['RA'])
    DEC.append(sampling['DEC'])